# MLflow を使用して Git ベースのアプリケーションのバージョンを追跡する

https://docs.databricks.com/aws/ja/mlflow3/genai/prompt-version-mgmt/version-tracking/track-application-versions-with-mlflow

**LoggedModel** は、Experiment 内で AI モデル、エージェント、または GenAI アプリケーションを表すファーストクラスのエンティティです。開発ライフサイクル全体を通じて、モデルアーティファクト、実行トレース、評価指標、メタデータの統合トラッキングを提供します。

## LoggedModel の主な機能
LoggedModel は次のものを接続する中心的なハブとして機能します:

- **モデルアーティファクト** と構成パラメータ
- 開発および本番環境からの **実行トレース**
- **評価指標** とパフォーマンス評価
- **バージョン履歴** とデプロイ追跡


## LoggedModel の利点
**統合トラッキング:** すべてのモデル関連アーティファクト、トレース、メトリクスが単一のエンティティの下に整理され、モデルの挙動とパフォーマンスを完全に可視化できます。

**自動トレースリンク:** アクティブモデルパターンを使用すると、すべての実行トレースが自動的に LoggedModel に関連付けられ、手動での追跡作業が不要になります。

**バージョン管理:** LoggedModel は、異なるモデル反復間での体系的なバージョン管理と比較をサポートし、データ駆動型のモデル選択を可能にします。

**評価統合:** 評価指標や結果が直接 LoggedModel にリンクされ、包括的なパフォーマンス評価が可能です。

In [0]:
%pip install --upgrade "mlflow[databricks]>=3.1.0" openai

%restart_python

In [0]:
import mlflow
from openai import OpenAI

# MLflowの自動ロギングを有効にして、アプリケーションにトレースを追加
mlflow.openai.autolog()

# トレースデコレータを使用してアプリケーションのエントリポイントをキャプチャ
@mlflow.trace
def my_app(model_input: str):
    # Notebook実行の資格情報を利用してDatabricks Foundation Model APIへのCredentialを取得
    mlflow_creds = mlflow.utils.databricks_utils.get_databricks_host_creds()
    client = OpenAI(
        api_key=mlflow_creds.token, base_url=f"{mlflow_creds.host}/serving-endpoints"
    )

    # この呼び出しは`mlflow.openai.autolog()`によって自動的に計測されます
    response = client.chat.completions.create(
        model="databricks-meta-llama-3-3-70b-instruct",
        messages=[
            {
                "role": "system",
                "content": "あなたは役に立つアシスタントです。",
            },
            {
                "role": "user",
                "content": model_input,
            },
        ],
    )
    return response.choices[0].message.content

# テスト実行
result = my_app(model_input="MLflowとは何ですか？")
print(result)

In [0]:
model_info = mlflow.pyfunc.log_model(
    name="my_app",
    python_model=my_app,
    pip_requirements=["openai"],
)

In [0]:
model = mlflow.pyfunc.load_model(model_info.model_uri)
prediction = model.predict("MLflowとは何ですか？")
print(prediction)


## バージョン管理

In [0]:
import mlflow
from openai import OpenAI

mlflow.openai.autolog()

@mlflow.trace
def my_app_revised(model_input: str):
    # Notebook実行の資格情報を利用してDatabricks Foundation Model APIへのCredentialを取得
    mlflow_creds = mlflow.utils.databricks_utils.get_databricks_host_creds()
    client = OpenAI(
        api_key=mlflow_creds.token, base_url=f"{mlflow_creds.host}/serving-endpoints"
    )

    # 利用するモデルをLlama3.3 70BからLlama4 に変更する
    response = client.chat.completions.create(
        model="databricks-llama-4-maverick",
        messages=[
            {
                "role": "system",
                "content": "あなたは役に立つアシスタントです。",
            },
            {
                "role": "user",
                "content": model_input,
            },
        ],
    )
    return response.choices[0].message.content

# テスト実行
result = my_app_revised(model_input="MLflowとは何ですか？")
print(result)

In [0]:
active_model_info = mlflow.set_active_model(name="my_app_dev")

In [0]:
# テスト実行２
my_app_revised(model_input="MLflowとは何ですか？")

In [0]:
# 新たなバージョンのモデルも併せて作成・設定
active_model_info = mlflow.set_active_model(name="my_app_dev2")

@mlflow.trace
def my_app_revise_ver2(model_input: str):
    mlflow_creds = mlflow.utils.databricks_utils.get_databricks_host_creds()
    client = OpenAI(
        api_key=mlflow_creds.token, base_url=f"{mlflow_creds.host}/serving-endpoints"
    )

    # システムプロンプトを変更
    response = client.chat.completions.create(
        model="databricks-llama-4-maverick",
        messages=[
            {
                "role": "system",
                "content": "あなたは役に立つアシスタントです。関西弁で面白おかしく回答してください。",
            },
            {
                "role": "user",
                "content": model_input,
            },
        ],
    )
    return response.choices[0].message.content

# テスト実行
result = my_app_revise_ver2(model_input="MLflowとは何ですか？")
print(result)

In [0]:
# Keep original imports
### NEW CODE
import subprocess

# Define your application and its version identifier
app_name = "customer_support_agent"

# Get current git commit hash for versioning
try:
    git_commit = (
        subprocess.check_output(["git", "rev-parse", "HEAD"])
        .decode("ascii")
        .strip()[:8]
    )
    version_identifier = f"git-{git_commit}"
except subprocess.CalledProcessError:
    version_identifier = "local-dev"  # Fallback if not in a git repo
logged_model_name = f"{app_name}-{version_identifier}"

# Set the active model context
active_model_info = mlflow.set_active_model(name=logged_model_name)
print(
    f"Active LoggedModel: '{active_model_info.name}', Model ID: '{active_model_info.model_id}'"
)

### END NEW CODE

### ORIGINAL CODE BELOW
### ...

In [0]:
app_params = {
    "llm": "gpt-4o-mini",
    "temperature": 0.7,
    "retrieval_strategy": "vector_search_v3",
}

# Log params
mlflow.log_model_params(model_id=active_model_info.model_id, params=app_params)

In [0]:
# These 2 invocations will be linked to the same LoggedModel
result = my_app(input="What is MLflow?")
print(result)

result = my_app(input="What is Databricks?")
print(result)

In [0]:

# Set the active model context
active_model_info = mlflow.set_active_model(name="new-name-set-manually")
print(
    f"Active LoggedModel: '{active_model_info.name}', Model ID: '{active_model_info.model_id}'"
)

app_params = {
    "llm": "gpt-4o",
    "temperature": 0.7,
    "retrieval_strategy": "vector_search_v4",
}

# Log params
mlflow.log_model_params(model_id=active_model_info.model_id, params=app_params)

# This will create a new LoggedModel
result = my_app(input="What is GenAI?")
print(result)